This is a condensed version of the 004 program. The sprk code which generates the dataset is moved to a library. In all other respects it is identical.

In [ ]:
hdfs_endpoint='hdfs://10.0.123.114:8020'
%env HDFS_PATH /user/dominospark/small_data
%env HADOOP_HOME /usr/lib/hadoop
%env HADOOP_CONF_DIR /etc/hadoop/conf
%env HADOOP_YARN_HOME /usr/lib/hadoop
%env HADOOP_MAPRED_HOME /usr/lib/hadoop
%env HADOOP_HDFS_HOME /usr/lib/hadoop-hdfs

%env SPARK_HOME /usr/lib/spark
%env SPARK_CONF_DIR /etc/spark/conf
#%env PYTHONPATH /opt/spark/python/lib/py4j-0.10.7-src.zip
%env PYSPARK_PYTHON /opt/conda/bin/python
%env PYSPARK_DRIVER_PYTHON /opt/conda/bin/python

In [ ]:
#Run spark on EMR in yarn cluster mode to create a dataset to use locally
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random
sys.path.insert(1, '/mnt/code/python/')
count=10

src_folder='/user/dominospark/large-data/'
dest_folder='/user/dominospark/smalldata-10'

!hdfs dfs -rmr '/user/dominospark/smalldata-10'

import filter_data_in_hdfs
args=[count,hdfs_endpoint,src_folder,dest_folder,'local[*]']
filter_data_in_hdfs.main(args)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random

In [ ]:
sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write") \
.master('local[*]') \
.config("fs.default.name", hdfs_endpoint) \
.getOrCreate()

In [ ]:
columns = StructType([ StructField("id", IntegerType(), True),
                       StructField("v1", IntegerType(), True),
                       StructField("v2", IntegerType(), True),
                       StructField("v3", IntegerType(), True) ])


df = sparkSession.read.csv(dest_folder, schema=columns)
df.show()
df.count()

In [ ]:
#Let us write to a dataset
ds_path = '/user/dominospark/my-pi'
!/mnt/code/scripts/my_hdfs.sh dfs -rmr '/user/dominospark/my-pi*'
df.write.csv(ds_path)
#Read it back
sparkSession.read.csv(ds_path).show()


In [ ]:
sparkSession.stop()

In [ ]:
!hdfs dfs -ls '/user/dominospark/smalldata-10'